In [76]:
import pathlib
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

In [65]:
%matplotlib inline

In [2]:
x = torch.rand(1)

In [4]:
x.size()

torch.Size([1])

In [5]:
t = torch.rand(10)

In [6]:
t.size()

torch.Size([10])

In [7]:
t

tensor([ 0.6161,  0.3998,  0.4109,  0.0569,  0.4600,  0.0237,  0.5524,
         0.9686,  0.8338,  0.4673])

In [8]:
x = torch.Tensor(2, 3)

In [12]:
#print(x)

In [10]:
x = torch.randn(2, 3)

In [11]:
print(x)

tensor([[ 1.0704, -0.3266, -0.0529],
        [ 2.4622,  0.5919,  2.1524]])


In [13]:
x = torch.ones(2, 3)
y = torch.ones(2, 3) * 2
x + y

tensor([[ 3.,  3.,  3.],
        [ 3.,  3.,  3.]])

In [14]:
y[:, 1]

tensor([ 2.,  2.])

In [15]:
y

tensor([[ 2.,  2.,  2.],
        [ 2.,  2.,  2.]])

In [16]:
y[1, 0] = 4

In [17]:
y

tensor([[ 2.,  2.,  2.],
        [ 4.,  2.,  2.]])

In [18]:
y[:, 1]

tensor([ 2.,  2.])

In [19]:
y[1, :]

tensor([ 4.,  2.,  2.])

In [20]:
y[0, 1] = 6

In [22]:
y

tensor([[ 2.,  6.,  2.],
        [ 4.,  2.,  2.]])

In [21]:
y[:, 1]

tensor([ 6.,  2.])

In [23]:
y[1, 1] = 0

In [24]:
y[:, 1]

tensor([ 6.,  0.])

In [25]:
y

tensor([[ 2.,  6.,  2.],
        [ 4.,  0.,  2.]])

In [26]:
z = torch.ones(2, 3) * 2

In [27]:
z

tensor([[ 2.,  2.,  2.],
        [ 2.,  2.,  2.]])

In [28]:
z[:, 1] = z[:, 1] + 1

In [29]:
z

tensor([[ 2.,  3.,  2.],
        [ 2.,  3.,  2.]])

In [30]:
print(z)

tensor([[ 2.,  3.,  2.],
        [ 2.,  3.,  2.]])


In [31]:
v = Variable(torch.ones(2, 2) * 2, requires_grad=True)

In [32]:
print(v)

tensor([[ 2.,  2.],
        [ 2.,  2.]])


In [33]:
t = 2 * (z * z) + 5 * z

In [34]:
print(t)

tensor([[ 18.,  33.,  18.],
        [ 18.,  33.,  18.]])


In [36]:
#t.backward(torch.ones(2, 2))

In [37]:
print(t.grad)

None


In [38]:
print(z.grad)

None


In [39]:
class Net(nn.Module):
    
    def __init__(self):
        # fc - fully connected layer
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

In [40]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [42]:
print(device)

cuda:0


In [43]:
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)

In [44]:
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


In [45]:
learning_rate = 0.01
batch_size = 200
epochs = 10
log_interval = 10

In [46]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.NLLLoss()

In [47]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size,
    shuffle=True
)

Processing...
Done!


In [48]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size,
    shuffle=True
)


In [49]:
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        data = data.view(-1, 28*28)
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0]
            )
        )

    # run a test loop
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    data = data.view(-1, 28 * 28)
    data = data.to(device)
    target = target.to(device)
    net_out = net(data)
    # sum up batch loss
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset))
    )

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.302897
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.170460
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.780523
Train Epoch: 0 [6000/60000 (10%)]	Loss: 1.170326
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.773083
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.518502
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.495792
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.464882
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.242585
Train Epoch: 0 [18000/60000 (30%)]	Loss: 0.439409
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.296521
Train Epoch: 0 [22000/60000 (37%)]	Loss: 0.264694
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.279281
Train Epoch: 0 [26000/60000 (43%)]	Loss: 0.340680
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.291895
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.310439
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.449987
Train Epoch: 0 [34000/60000 (57%)]	Loss: 0.336213
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.293228
Train Epoch: 0 [38000/60000 (63%)]	Loss: 0.284978
Train Epoch

Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.047625
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.085780
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.060010
Train Epoch: 5 [38000/60000 (63%)]	Loss: 0.047949
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.074389
Train Epoch: 5 [42000/60000 (70%)]	Loss: 0.099181
Train Epoch: 5 [44000/60000 (73%)]	Loss: 0.091449
Train Epoch: 5 [46000/60000 (77%)]	Loss: 0.105185
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.129122
Train Epoch: 5 [50000/60000 (83%)]	Loss: 0.066544
Train Epoch: 5 [52000/60000 (87%)]	Loss: 0.053156
Train Epoch: 5 [54000/60000 (90%)]	Loss: 0.110336
Train Epoch: 5 [56000/60000 (93%)]	Loss: 0.100817
Train Epoch: 5 [58000/60000 (97%)]	Loss: 0.050327
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.052438
Train Epoch: 6 [2000/60000 (3%)]	Loss: 0.033748
Train Epoch: 6 [4000/60000 (7%)]	Loss: 0.037035
Train Epoch: 6 [6000/60000 (10%)]	Loss: 0.072417
Train Epoch: 6 [8000/60000 (13%)]	Loss: 0.074731
Train Epoch: 6 [10000/60000 (17%)]	Loss: 0.085176
Train Epoch


Test set: Average loss: 0.0000, Accuracy: 6260/10000 (62%)


Test set: Average loss: 0.0000, Accuracy: 6457/10000 (64%)


Test set: Average loss: 0.0000, Accuracy: 6651/10000 (66%)


Test set: Average loss: 0.0000, Accuracy: 6849/10000 (68%)


Test set: Average loss: 0.0000, Accuracy: 7043/10000 (70%)


Test set: Average loss: 0.0000, Accuracy: 7238/10000 (72%)


Test set: Average loss: 0.0000, Accuracy: 7435/10000 (74%)


Test set: Average loss: 0.0000, Accuracy: 7632/10000 (76%)


Test set: Average loss: 0.0000, Accuracy: 7828/10000 (78%)


Test set: Average loss: 0.0000, Accuracy: 8024/10000 (80%)


Test set: Average loss: 0.0000, Accuracy: 8218/10000 (82%)


Test set: Average loss: 0.0000, Accuracy: 8411/10000 (84%)


Test set: Average loss: 0.0000, Accuracy: 8607/10000 (86%)


Test set: Average loss: 0.0000, Accuracy: 8807/10000 (88%)


Test set: Average loss: 0.0000, Accuracy: 9003/10000 (90%)


Test set: Average loss: 0.0000, Accuracy: 9199/10000 (91%)


Test set: Average loss:

In [50]:
a = torch.rand(10000, 10000)
b = torch.rand(10000, 10000)

In [52]:
%time a.matmul(b)

CPU times: user 1min 21s, sys: 644 ms, total: 1min 22s
Wall time: 4.65 s


tensor([[ 2506.7471,  2495.0217,  2498.4839,  ...,  2529.3350,
          2506.5103,  2484.4294],
        [ 2501.4666,  2490.8452,  2501.4294,  ...,  2515.9888,
          2498.2966,  2472.1545],
        [ 2512.6753,  2509.8667,  2492.7029,  ...,  2524.4434,
          2503.6060,  2469.2080],
        ...,
        [ 2499.3472,  2510.8064,  2500.6414,  ...,  2522.5752,
          2488.1218,  2479.3989],
        [ 2498.7332,  2515.1230,  2482.9761,  ...,  2525.3154,
          2502.9756,  2485.4617],
        [ 2502.6255,  2479.3650,  2496.1970,  ...,  2532.7366,
          2495.4912,  2488.6138]])

In [53]:
a = a.cuda()
b = b.cuda()

In [54]:
%time a.matmul(b)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.41 ms


tensor([[ 2506.7427,  2495.0146,  2498.4790,  ...,  2529.3384,
          2506.5112,  2484.4333],
        [ 2501.4583,  2490.8447,  2501.4258,  ...,  2515.9851,
          2498.2954,  2472.1621],
        [ 2512.6772,  2509.8589,  2492.7065,  ...,  2524.4436,
          2503.6018,  2469.2053],
        ...,
        [ 2499.3435,  2510.8057,  2500.6392,  ...,  2522.5706,
          2488.1260,  2479.3970],
        [ 2498.7275,  2515.1238,  2482.9858,  ...,  2525.3220,
          2502.9827,  2485.4604],
        [ 2502.6230,  2479.3679,  2496.1943,  ...,  2532.7329,
          2495.4917,  2488.6091]], device='cuda:0')

In [55]:
xx = Variable(torch.ones(2, 2), requires_grad=True)

In [56]:
yy = xx.mean()

In [58]:
xx.grad

In [59]:
yy.backward()

In [60]:
xx.grad

tensor([[ 0.2500,  0.2500],
        [ 0.2500,  0.2500]])

In [61]:
xx.data

tensor([[ 1.,  1.],
        [ 1.,  1.]])

In [62]:
xx.grad_fn

In [63]:
yy.grad_fn

In [70]:
def get_data():
    train_X = np.asarray([3.3, 4.4, 5.5, 6.71, 6.93, 4.168, 9.779, 6.182, 7.59, 2.167, 7.042,
                         10.791, 5.313, 7.997, 5.654, 9.27, 3.1])
    train_Y = np.asarray([1.7, 2.76, 2.09, 3.19, 1.694, 1.573, 3.336, 2.596, 2.53, 1.221, 2.827,
                         3.465, 1.65, 2.904, 2.42, 2.94, 1.3])
    #print(train_X.shape)
    #print(train_Y.shape)
    dtype = torch.FloatTensor
    X = Variable(torch.from_numpy(train_X).type(dtype), requires_grad=False).view(17, 1)
    y = Variable(torch.from_numpy(train_Y).type(dtype), requires_grad=False)
    return X, y

In [71]:
def get_weights():
    w = Variable(torch.randn(1), requires_grad=True)
    b = Variable(torch.randn(1), requires_grad=True)
    return w, b

In [72]:
def simple_network(x):
    y_pred = torch.matmul(x, w) + b
    return y_pred

In [73]:
def loss_fn(y, y_pred):
    loss = (y_pred - y).pow(2).sum()
    for param in [w, b]:
        if not param.grad is None:
            param.grad.data.zero_()
    loss.backward()
    return loss.data[0]

In [122]:
class DogsAndCatsDataset(torch.utils.data.Dataset):
    
    def __init__(self, root_dir, size=(224, 224)):
        #self.files = pathlib.Path.glob(root_dir, '*.jpg')
        self.files = os.listdir(root_dir)
        self.size = size
        self.root_dir = root_dir
        #print(len(self.files))
        #pass
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img = np.asanyarray(Image.open(os.path.join(self.root_dir, self.files[idx])).resize(self.size))
        img_name = self.files[idx]
        #label = self.files[idx].split('/')[-2]
        #label = img_name.split('/')[-2]
        label = img_name.split('.')[0]
        #print(label)
        #print(type(label))
        return img, label

In [123]:
dogs_cats_dset = DogsAndCatsDataset('dogscats/train/')

In [124]:
type(dogs_cats_dset)
dogs_cats_dset.__dict__
dir(dogs_cats_dset)
dogs_cats_dset.__dir__()
dogs_cats_dset.__len__()

25000

In [125]:
dataloader = DataLoader(dogs_cats_dset, batch_size=32, num_workers=2)

In [127]:
for i, (img, label) in enumerate(dataloader):
    print(i)
    print(type(img), type(label))
    print(len(img), len(label))
    #pass

0
<class 'torch.Tensor'> <class 'tuple'>
32 32
1
<class 'torch.Tensor'> <class 'tuple'>
32 32
2
<class 'torch.Tensor'> <class 'tuple'>
32 32
3
<class 'torch.Tensor'> <class 'tuple'>
32 32
4
<class 'torch.Tensor'> <class 'tuple'>
32 32
5
<class 'torch.Tensor'> <class 'tuple'>
32 32
6
<class 'torch.Tensor'> <class 'tuple'>
32 32
7
<class 'torch.Tensor'> <class 'tuple'>
32 32
8
<class 'torch.Tensor'> <class 'tuple'>
32 32
9
<class 'torch.Tensor'> <class 'tuple'>
32 32
10
<class 'torch.Tensor'> <class 'tuple'>
32 32
11
<class 'torch.Tensor'> <class 'tuple'>
32 32
12
<class 'torch.Tensor'> <class 'tuple'>
32 32
13
<class 'torch.Tensor'> <class 'tuple'>
32 32
14
<class 'torch.Tensor'> <class 'tuple'>
32 32
15
<class 'torch.Tensor'> <class 'tuple'>
32 32
16
<class 'torch.Tensor'> <class 'tuple'>
32 32
17
<class 'torch.Tensor'> <class 'tuple'>
32 32
18
<class 'torch.Tensor'> <class 'tuple'>
32 32
19
<class 'torch.Tensor'> <class 'tuple'>
32 32
20
<class 'torch.Tensor'> <class 'tuple'>
32 32
21

171
<class 'torch.Tensor'> <class 'tuple'>
32 32
172
<class 'torch.Tensor'> <class 'tuple'>
32 32
173
<class 'torch.Tensor'> <class 'tuple'>
32 32
174
<class 'torch.Tensor'> <class 'tuple'>
32 32
175
<class 'torch.Tensor'> <class 'tuple'>
32 32
176
<class 'torch.Tensor'> <class 'tuple'>
32 32
177
<class 'torch.Tensor'> <class 'tuple'>
32 32
178
<class 'torch.Tensor'> <class 'tuple'>
32 32
179
<class 'torch.Tensor'> <class 'tuple'>
32 32
180
<class 'torch.Tensor'> <class 'tuple'>
32 32
181
<class 'torch.Tensor'> <class 'tuple'>
32 32
182
<class 'torch.Tensor'> <class 'tuple'>
32 32
183
<class 'torch.Tensor'> <class 'tuple'>
32 32
184
<class 'torch.Tensor'> <class 'tuple'>
32 32
185
<class 'torch.Tensor'> <class 'tuple'>
32 32
186
<class 'torch.Tensor'> <class 'tuple'>
32 32
187
<class 'torch.Tensor'> <class 'tuple'>
32 32
188
<class 'torch.Tensor'> <class 'tuple'>
32 32
189
<class 'torch.Tensor'> <class 'tuple'>
32 32
190
<class 'torch.Tensor'> <class 'tuple'>
32 32
191
<class 'torch.Te

340
<class 'torch.Tensor'> <class 'tuple'>
32 32
341
<class 'torch.Tensor'> <class 'tuple'>
32 32
342
<class 'torch.Tensor'> <class 'tuple'>
32 32
343
<class 'torch.Tensor'> <class 'tuple'>
32 32
344
<class 'torch.Tensor'> <class 'tuple'>
32 32
345
<class 'torch.Tensor'> <class 'tuple'>
32 32
346
<class 'torch.Tensor'> <class 'tuple'>
32 32
347
<class 'torch.Tensor'> <class 'tuple'>
32 32
348
<class 'torch.Tensor'> <class 'tuple'>
32 32
349
<class 'torch.Tensor'> <class 'tuple'>
32 32
350
<class 'torch.Tensor'> <class 'tuple'>
32 32
351
<class 'torch.Tensor'> <class 'tuple'>
32 32
352
<class 'torch.Tensor'> <class 'tuple'>
32 32
353
<class 'torch.Tensor'> <class 'tuple'>
32 32
354
<class 'torch.Tensor'> <class 'tuple'>
32 32
355
<class 'torch.Tensor'> <class 'tuple'>
32 32
356
<class 'torch.Tensor'> <class 'tuple'>
32 32
357
<class 'torch.Tensor'> <class 'tuple'>
32 32
358
<class 'torch.Tensor'> <class 'tuple'>
32 32
359
<class 'torch.Tensor'> <class 'tuple'>
32 32
360
<class 'torch.Te

508
<class 'torch.Tensor'> <class 'tuple'>
32 32
509
<class 'torch.Tensor'> <class 'tuple'>
32 32
510
<class 'torch.Tensor'> <class 'tuple'>
32 32
511
<class 'torch.Tensor'> <class 'tuple'>
32 32
512
<class 'torch.Tensor'> <class 'tuple'>
32 32
513
<class 'torch.Tensor'> <class 'tuple'>
32 32
514
<class 'torch.Tensor'> <class 'tuple'>
32 32
515
<class 'torch.Tensor'> <class 'tuple'>
32 32
516
<class 'torch.Tensor'> <class 'tuple'>
32 32
517
<class 'torch.Tensor'> <class 'tuple'>
32 32
518
<class 'torch.Tensor'> <class 'tuple'>
32 32
519
<class 'torch.Tensor'> <class 'tuple'>
32 32
520
<class 'torch.Tensor'> <class 'tuple'>
32 32
521
<class 'torch.Tensor'> <class 'tuple'>
32 32
522
<class 'torch.Tensor'> <class 'tuple'>
32 32
523
<class 'torch.Tensor'> <class 'tuple'>
32 32
524
<class 'torch.Tensor'> <class 'tuple'>
32 32
525
<class 'torch.Tensor'> <class 'tuple'>
32 32
526
<class 'torch.Tensor'> <class 'tuple'>
32 32
527
<class 'torch.Tensor'> <class 'tuple'>
32 32
528
<class 'torch.Te

679
<class 'torch.Tensor'> <class 'tuple'>
32 32
680
<class 'torch.Tensor'> <class 'tuple'>
32 32
681
<class 'torch.Tensor'> <class 'tuple'>
32 32
682
<class 'torch.Tensor'> <class 'tuple'>
32 32
683
<class 'torch.Tensor'> <class 'tuple'>
32 32
684
<class 'torch.Tensor'> <class 'tuple'>
32 32
685
<class 'torch.Tensor'> <class 'tuple'>
32 32
686
<class 'torch.Tensor'> <class 'tuple'>
32 32
687
<class 'torch.Tensor'> <class 'tuple'>
32 32
688
<class 'torch.Tensor'> <class 'tuple'>
32 32
689
<class 'torch.Tensor'> <class 'tuple'>
32 32
690
<class 'torch.Tensor'> <class 'tuple'>
32 32
691
<class 'torch.Tensor'> <class 'tuple'>
32 32
692
<class 'torch.Tensor'> <class 'tuple'>
32 32
693
<class 'torch.Tensor'> <class 'tuple'>
32 32
694
<class 'torch.Tensor'> <class 'tuple'>
32 32
695
<class 'torch.Tensor'> <class 'tuple'>
32 32
696
<class 'torch.Tensor'> <class 'tuple'>
32 32
697
<class 'torch.Tensor'> <class 'tuple'>
32 32
698
<class 'torch.Tensor'> <class 'tuple'>
32 32
699
<class 'torch.Te

In [128]:
781 * 32 + 8

25000